# Create Deduplicated CSV File
This notebook processes the input CSV file, removes duplicate `query_id` entries, and creates a new CSV file with averaged values for duplicates.

In [45]:
import pandas as pd
import os

In [46]:
# Define file paths
# input_path = "tpc-ds/result_log/result_log_20GB/time_3/query_sys_params.csv"
# average_path = "tpc-ds/result_log/result_log_20GB/time_3/average_query_sys_params.csv"
# worst_path = "tpc-ds/result_log/result_log_20GB/time_3/worst_query_sys_params.csv"
# query_times_path = "tpc-ds/result_log/result_log_20GB/time_3/query_times.csv"
input_path = "tpc-h/result_log/result_log_20GB/time_3/query_sys_params.csv"
average_path = "tpc-h/result_log/result_log_20GB/time_3/average_query_sys_params.csv"
worst_path = "tpc-h/result_log/result_log_20GB/time_3/worst_query_sys_params.csv"
query_times_path = "tpc-h/result_log/result_log_20GB/time_3/query_times.csv"

In [47]:
# Read the input CSV file - don't skip the first row since it contains headers
data = pd.read_csv(input_path)

# Clean column names (remove spaces)
data.columns = [col.strip() for col in data.columns]

# Print column names to see what's available
print("Available columns:", data.columns.tolist())

# Convert columns to numeric types
data['cpu_used(%)'] = pd.to_numeric(data['cpu_used(%)'], errors='coerce')
data['ram_used(gb)'] = pd.to_numeric(data['ram_used(gb)'], errors='coerce')

# Now group by query_id and calculate the average for duplicates
average_data = data.groupby('query_id', as_index=False).agg({
    'cpu_used(%)': 'mean',
    'ram_used(gb)': 'mean'
})

# Calculate worst-case (maximum) values
worst_data = data.groupby('query_id', as_index=False).agg({
    'cpu_used(%)': 'max',
    'ram_used(gb)': 'max'
})

# Round to 2 decimal places
average_data = average_data.round(2)
worst_data = worst_data.round(2)

# Print the first few rows of each to verify
print("Average data:")
print(average_data.head())
print("\nWorst case data:")
print(worst_data.head())

Available columns: ['query_id', 'cpu_used(%)', 'ram_used(gb)']
Average data:
   query_id  cpu_used(%)  ram_used(gb)
0         1        88.62          1.22
1         2        46.70          1.04
2         3        78.99          1.41
3         4        79.00          1.30
4         5        74.58          1.47

Worst case data:
   query_id  cpu_used(%)  ram_used(gb)
0         1         90.9          1.73
1         2         76.2          1.05
2         3         88.1          1.85
3         4         89.4          1.63
4         5         89.2          2.06


In [48]:
# Load the query times data

query_times = pd.read_csv(query_times_path)

# Clean column names
query_times.columns = [col.strip() for col in query_times.columns]

# Convert real_time from seconds to milliseconds
query_times['time(ms)'] = query_times['real_time(s)'] * 1000

# Group by query_id and calculate the average for duplicates in query_times
deduplicated_times = query_times.groupby('query_id', as_index=False).agg({
    'real_time(s)': 'mean',
    'user_time(s)': 'mean',
    'sys_time(s)': 'mean',
    'time(ms)': 'mean'
})

# Merge the deduplicated system parameters with the deduplicated times
merged_average_data = pd.merge(average_data, deduplicated_times[['query_id', 'time(ms)']], 
                      on='query_id', how='left')
worst_average_data = pd.merge(worst_data, deduplicated_times[['query_id', 'time(ms)']], 
                      on='query_id', how='left')
# Save the merged data to a new CSV file
merged_average_data.to_csv(average_path, index=False)
worst_average_data.to_csv(worst_path, index=False)